<h2><center> MACHINE LEARNING PROJECT - N. 6 </center></h2>

<h3><center> Subscript nr. 4 - Advanced Techniques </center></h3>

This notebook contains some alternative models proposed by us to complete the analysis.

Some of the libraries used are <i>sklearn</i> and <i>imblearn</i>.

### Random over sampling Tree

A decision tree has been performed on the balanced data by using the function. 

In [1]:
def ros_tree(new_df, ros_df, clf, dict_font):
   
    X_train_ros, _, y_train_ros, _ = train_test_split(
                        ros_df[["Latitude", "TempCels", "SpecHum", "RelHum", "AbsHum"]],
                        ros_df["Cases"], test_size = 0.2, random_state = 1)
    
    _, X_test, _, y_test = train_test_split(
                        new_df[["Latitude", "TempCels", "SpecHum", "RelHum", "AbsHum"]],
                        new_df["Cases"], test_size = 0.2, random_state = 1)
    
    clf = clf.fit(X_train_ros, y_train_ros)
    y_pred = clf.predict(X_test)

    print("Test MSE: ", round( metrics.mean_squared_error(y_test, y_pred) , 3))
    print("Test MAPE: ", round( metrics.mean_absolute_percentage_error(y_test, y_pred) , 3))
    print("Test MAE: ", round( metrics.median_absolute_error(y_test, y_pred) , 3))
    #print("The median is more robust to outliers. The percentage error is high.")
    
    fig = plt.figure(figsize = np.array(dict_font["fig_size"]) * [2, 2])
    plt.rcParams['figure.dpi'] = dict_font["fig_dpi"] * 2
    t = tree.plot_tree(clf, filled = True, feature_names = X_train_ros.columns.to_list(),
    rounded = True, proportion = False)
    return

### Random over sampling regression

A linear regression has been used to understand if the covid cases (from random oversampling dataset) are linearly related to the other variables. 
The function performs the summary of the model. 

In [2]:
def ros_reg(new_df, ros_df):
    
    X_train_ros, _, y_train_ros, _ = train_test_split(
                    ros_df[["Latitude", "TempCels", "SpecHum", "RelHum", "AbsHum"]], 
                    ros_df["Cases"], test_size = 0.2, random_state = 1) 
    
    _, X_test, _, y_test = train_test_split(
                    new_df[["Latitude", "TempCels", "SpecHum", "RelHum", "AbsHum"]], 
                    new_df["Cases"], test_size = 0.2, random_state = 1) 

    # X_train_ros_2 = sm.add_constant(X_train_ros) # column of ones to estimate intercept
    X_train_ros_2 = X_train_ros.copy()
    X_train_ros_2["Const"] = 1 # add_constant may be not well-suited for pd.dataframes
    model2 = sm.OLS(y_train_ros, X_train_ros_2).fit()
    print(model2.summary())

    model = LinearRegression()
    model.fit(X_train_ros, y_train_ros)
    y_pred = model.predict(X_test)
    print('MAE:', round(metrics.mean_absolute_error(y_test, y_pred), 3) )
    
    return 

### K-Means

#### Elbow method

The same is made with another kind of rule, the elbow. 

In [10]:
def kmeans_elbow(scaled_df, dict_font):
    elbow_scores = {}
    K = range(1, 25) # 1 clusters is meaningless
    
    for k in K:
        model_kmeans_k = KMeans(n_clusters = k)
        model_kmeans_k.fit(scaled_df)
        labels_k = model_kmeans_k.labels_  
        elbow_scores[k] = model_kmeans_k.inertia_

    plt.figure(figsize = dict_font["fig_size"])
    plt.plot(K, elbow_scores.values(), marker = "x", markersize = dict_font["font_txt"] * 0.5)
    plt.xlabel('k')
    plt.ylabel('Elbow score')
    plt.title('The Elbow Method')
    plt.xticks(K)
    plt.plot(3, elbow_scores[3], "oy", markersize = dict_font["font_xy"] * 0.8, label = "Solution A", alpha = 0.75)
    plt.plot(4, elbow_scores[4], "og", markersize = dict_font["font_xy"] * 0.8, label = "Solution B", alpha = 0.75)
    plt.legend(title = 'Nr. Cluster', loc='best', fontsize = dict_font["font_txt"] * 0.8)

    return plt.show()

#### Silhouette plot

A function to perform the choice on the number of clusters k is implemented. Furthermore the graph to plot the best k is presented.

$$ \frac{b - a}{\max(a,b)} $$

* a: mean intra-cluster distance
* b: mean nearest- cluster distance


In [3]:
def kmeans_silhouette(scaled_df, dict_font):
    k_to_test = range(2, 25) 
    silhouette_scores = {}

    for k in k_to_test:
        model_kmeans_k = KMeans(n_clusters = k)
        model_kmeans_k.fit(scaled_df)
        labels_k = model_kmeans_k.labels_
        silhouette_scores[k] = metrics.silhouette_score(scaled_df, labels_k) # mean over whole samples

    plt.rcParams['figure.dpi'] = dict_font["fig_dpi"] * 1.2
    plt.rcParams['figure.figsize'] = np.array(dict_font["fig_size"]) * [0.8,0.7]
    
    plt.plot(silhouette_scores.keys(), silhouette_scores.values(), marker = "x", markersize = dict_font["font_txt"] * 0.5)
    
    # coords = max(zip(silhouette_scores.values(), silhouette_scores.keys()))
    coords = max(silhouette_scores, key = silhouette_scores.get), max(silhouette_scores.values())
    plt.plot(coords[0], coords[1], "or", markersize = dict_font["font_xy"] * 0.8, label = "Optimal Number")
    
    plt.legend(title = 'Nr. Cluster', loc = 'best', fontsize = dict_font["font_txt"] * 0.8)
    plt.xticks(k_to_test)
    plt.title("Silhouette analysis for optimal k", fontsize = dict_font["font_title"])
    plt.xlabel("k", fontsize = dict_font["font_xy"])
    plt.ylabel("Silhouette Score", fontsize = dict_font["font_xy"])
    plt.xticks(fontsize = dict_font["font_txt"] * 0.9)
    plt.yticks(fontsize = dict_font["font_txt"] * 0.9)
    
    return plt.show(), silhouette_scores

#### K-Means plot

Plot of clusters choosen with silhouette method.

In [5]:
def kmeans_plot(scaled_df, silhouette_scores, dict_font):
    kmeans_model = KMeans(n_clusters = max(silhouette_scores, key = silhouette_scores.get) )
    kmeans_model.fit(scaled_df)
    centroids = kmeans_model.cluster_centers_

    city_clust = scaled_df.copy()
    city_clust["Cluster"] = kmeans_model.labels_ # distinguish colors
    city_clust["City"] = new_df["City"]

    plt.rcParams['figure.dpi'] = dict_font["fig_dpi"] * 1.2
    plt.rcParams['figure.figsize'] = dict_font["fig_size"]
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = np.array(dict_font["fig_size"])*[1.1,1.1])

    sns.scatterplot(x = "TempCels", y = "Latitude", data = city_clust, hue = "Cluster", palette = "Accent", ax = ax1, legend = False)
    sns.scatterplot(x = "TempCels", y = "RelHum", data = city_clust, hue = "Cluster", palette = "Accent", ax = ax2, legend = False)
    sns.scatterplot(x = "SpecHum", y = "Latitude", data = city_clust, hue = "Cluster", palette = "Accent", ax = ax3, legend = False)
    sns.scatterplot(x = "SpecHum", y = "TempCels", data = city_clust, hue = "Cluster", palette = "Accent", ax = ax4, legend = False)

    return plt.show()

### Ridge regression

First, for a set of alpha the model is fitted; 
then by using the best alpha (i.e., the one which produces lowest MSE), the final model is printed. 


In [6]:
def ridge(X_train, X_test, y_train, y_test):
    ridge = Ridge(fit_intercept = True)
    coefs = []
    mse_list = []

    alphas = 10**np.linspace(start = 10, stop = -2, num = 250)*0.5

    for a in alphas:
        ridge.set_params(alpha = a)
        ridge.fit(X_train, y_train)
        coefs.append(ridge.coef_)
        mse_list.append(metrics.mean_squared_error(y_test, ridge.predict(X_test)))

    alpha_best = alphas[np.argmin(mse_list)]
    print("Penalties tested:", len(alphas))
    print("Numbers of predictors:", np.shape(coefs)[1])
    print("Best alpha: ", round(alpha_best, 4))
    
    ridge_best = Ridge(alpha = alpha_best, fit_intercept = True)
    ridge_best.fit(X_train, y_train)
    y_hat = ridge_best.predict(X_test)
    print("Test MSE: ", round(metrics.mean_squared_error(y_test, y_hat), 3), sep = "")

    print("Intercept", ": ", "\u03B2", 0, " = ",  round(ridge_best.intercept_, 3), sep = "")
    for i in range(len(ridge_best.feature_names_in_)):
        print(ridge_best.feature_names_in_[i], ": ", "\u03B2", i+1, " = " ,
              round(ridge_best.coef_[i], 3), sep = "")
    
    return

### Lasso

In [7]:
def lasso(X_train, X_test, y_train, y_test):
    lasso = Lasso(max_iter = 10000, fit_intercept = True)
    coefs = []
    mse_list = []

    alphas = 10**np.linspace(start = 10, stop = -2, num = 250)*0.5

    for a in alphas:
        lasso.set_params(alpha = a)
        lasso.fit(X_train, y_train)
        coefs.append(lasso.coef_)
        mse_list.append(metrics.mean_squared_error(y_test, lasso.predict(X_test)))

    alpha_best = alphas[np.argmin(mse_list)]
    print("Penalties tested:", np.shape(coefs)[0])
    print("Numbers of predictors:", np.shape(coefs)[1])
    print("Best alpha: ", round(alpha_best, 4))
    lasso_best = Lasso(alpha = alpha_best, fit_intercept=True)
    lasso_best.fit(X_train, y_train)
    y_hat = lasso_best.predict(X_test)
    print("MSE: ", round(metrics.mean_squared_error(y_test, y_hat), 3), sep = "")
    
    print("Intercept", ": ", "\u03B2", 0,  " = ", round(lasso_best.intercept_, 3), sep = "")
    for i in range(len(lasso_best.feature_names_in_)):
          print(lasso_best.feature_names_in_[i], ": " ,"\u03B2", i+1, " = ", 
                round(lasso_best.coef_[i], 3), sep = "")
    return 

### Feature selection

In [11]:
def feature_selection(X_train, X_test, y_train, y_test):
    models_best = pd.DataFrame( columns = ["RSS", "model"] ) # empty

    X_train["Const"] = 1
    X_test["Const"] = 1

    for i in range(1,len(X_train.columns) + 1): # + 1 to include the model with all vars as regressors
        models_best.loc[i] = get_best(i, X_train, X_test, y_train, y_test)
    
    fs_coeff = pd.DataFrame({
        "Nr Coef": [(i+1) for i in range(models_best.shape[0])],
        "RSS": models_best["RSS"] })
    
    #del X_train["Const"]
    #del X_test["Const"]
    
    return fs_coeff, models_best

In order to perform best subset selection it is required to identify the best model that contains a given number of predictors. We'll define a helper function to outputs the best set of variables for each model size.

In [ ]:
def get_best(i, X_train, X_test, y_train, y_test):  
    results = []
    for var_subset in itertools.combinations(X_train.columns, i):
        results.append(process_subset(var_subset, X_train, X_test, y_train, y_test)) 
                                        # returns dict with model and its RSS
    
    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].argmin()]
    
    return best_model

This function fits a model for a certain subset of predictors. Instead of evaluating the MSE, the RSS is computed.

In [ ]:
def process_subset(var_subset, X_train, X_test, y_train, y_test):
    
    model = sm.OLS(y_train, X_train[list(var_subset)], hasconst = False) #constant is included
    regr = model.fit()
    y_hat = regr.predict(X_test[list(var_subset)])
    RSS = ((y_test - y_hat) ** 2).sum()
    
    return {"model" : regr, "RSS" : RSS}

#### Plot for feature selection

In [11]:
def plot_fs(models_best, dict_font):
    plt.rcParams['figure.dpi'] = dict_font["fig_dpi"]
    plt.rcParams['figure.figsize'] = dict_font["fig_size"]

    plt.subplot(1, 2, 1)

    plt.plot(models_best["RSS"], marker = "x", markersize = 12)
    plt.plot(models_best["RSS"].argmin()+1, models_best["RSS"].min(), "or", markersize = dict_font["font_xy"] * 0.8,
            label = "Optimal Number")
    leg1 = plt.legend(title = 'Predictors', loc='best', fontsize = dict_font["font_txt"] * 0.75)
    leg1.set_title(title = 'Predictors', prop = {'size': dict_font["font_title"] * 0.8})
    plt.xlabel('Number of Predictors', fontsize = dict_font["font_xy"])
    plt.ylabel('RSS', fontsize = dict_font["font_xy"])
    plt.xticks(ticks = [(i+1) for i in range(models_best.shape[0])], fontsize = dict_font["font_xy"], rotation=0)
    plt.yticks(fontsize = dict_font["font_xy"])

    aic = models_best.apply(lambda row: row[1].aic, axis=1)

    plt.subplot(1,2,2)
    plt.plot(aic, marker = "x", markersize = 12)
    plt.plot(aic.argmin()+1, aic.min(), "or", markersize = dict_font["font_xy"] * 0.8, label = "Optimal Number")
    plt.legend(title = 'Predictors', loc='best', fontsize = dict_font["font_txt"] * 0.8)
    leg2 = plt.legend(title = 'Predictors', loc='best', fontsize = dict_font["font_txt"] * 0.75)
    leg2.set_title(title = 'Predictors', prop = {'size': dict_font["font_title"] * 0.8})
    plt.yticks(fontsize = dict_font["font_xy"])
    plt.xticks(ticks = [(i+1) for i in range(models_best.shape[0])], fontsize = dict_font["font_xy"], rotation=0)

    plt.xlabel('Number of Predictors', fontsize = dict_font["font_xy"])
    plt.ylabel('AIC', fontsize = dict_font["font_xy"])
    plt.tight_layout(pad = 3)
    plt.show()